# Test for visit_skymaps

In [ ]:
from rubin_sim.data import get_baseline

from schedview.plot.visit_skymaps import VisitMapBuilder
from schedview.collect import read_opsim
from uranography.api import ArmillarySphere, Planisphere
import bokeh.io
import numpy as np
from rubin_scheduler.scheduler.utils import get_current_footprint

In [ ]:
bokeh.io.output_notebook()

In [ ]:
night = 365
nside = 64


In [ ]:
baseline_visits = read_opsim(get_baseline())
present_visit_columns = [c for c in VisitMapBuilder.visit_columns if c in baseline_visits.columns]
visits = baseline_visits.reset_index().set_index('night').loc[[night, night+1, night+2], present_visit_columns + ['observationId']].reset_index().set_index('observationId').copy()
visits.describe().T

In [ ]:
footprint_depth_by_band, footprint_regions = get_current_footprint(nside)
bands_in_footprint = tuple(footprint_depth_by_band.dtype.fields.keys())
footprint_depth = footprint_depth_by_band[bands_in_footprint[0]]
for band in bands_in_footprint[1:]:
    footprint_depth = footprint_depth + footprint_depth_by_band[band]

In [ ]:
builder = (
VisitMapBuilder(
    visits,
    mjd=visits['observationStartMJD'].max(),
    map_classes=[ArmillarySphere, Planisphere],
    figure_kwargs={'match_aspect': True, 'border_fill_color': 'black', 'background_fill_color': 'black'}
    )
    .add_footprint_outlines(footprint_regions)
#    .add_footprint(footprint_depth)
    .hide_horizon_sliders()
    .make_up_north()
    .add_eq_sliders()
    .add_graticules()
    .add_ecliptic()
    .add_galactic_plane()
    .add_mjd_slider(
            start=visits['observationStartMJD'].min(),
            end=visits['observationStartMJD'].max()
    )
    .add_datetime_slider()
    .hide_mjd_slider()
    .hide_future_visits()
    .highlight_recent_visits()
    .add_body('sun', size=15, color='yellow', alpha=1.0)
    .add_body('moon', size=15, color='orange', alpha=0.8)
    .add_horizon()
    .add_horizon(zd=70, color='red')
    .add_hovertext()
)
viewable = builder.build()

In [ ]:
bokeh.io.show(viewable)